In [36]:
from math import floor

class field(object):
    def __init__(self, elementary_str, contractions, symbol):
        self.elementary_str = elementary_str #Stores list of N symbols for elementary field factors phi and dphi
        self.elementary = []
        self.contractions = contractions #Stores the indices with respect to which successive elementary factors are linked
                                #e.g., [[0,1],[1,0],[0,1]] for phi^i1 dphi_u1^i1 dphi_u1^i2 phi^i2, [[1,1]] for dphi_u^i dphi_u^i. For N 
                                #elementary fields, N-1 links.
        self.symbol = symbol
        self.generated_terms = []
    def __str__(self):
        return 'elementary_str: ' + str(self.elementary_str) + '\n' \
            + 'contractions: ' + str(self.contractions) + '\n' \
            + 'symbol: ' + str(self.symbol) + '\n' \
            + 'massDim: ' + str(self.massDim()) + '\n' \
            + 'LORrank: ' + str(self.LORrank()) + '\n' \
            + 'ONrank: ' + str(self.ONrank()) 
            
    def __eq__(self, other):
        eq = (self.elementary_str == other.elementary_str) & (self.contractions == other.contractions)
        return eq
    
    #Define get and set functions
    def get_elementary_str(self):
        return self.elementary_str
    def get_elementary(self):
        return self.elementary
    def get_contractions(self):
        return self.contractions
    def get_symbol(self):
        return self.symbol
    
    def set_elementary_str(self, elementary_str):
        self.elementary_str = elementary_str
    def set_elementary(self): #should run this to set list of constituent fields
        for symbol in self.elementary_str:
            if symbol == 'P':
                field1 = field(['P'],[], 'P')
                self.elementary.append(field1)
            elif symbol == 'DP':
                field1_deriv = field(['DP'],[], 'DP')
                self.elementary.append(field1_deriv)
            else:
                print('Error: Symbol must be P or DP.')
    def set_contractions(self, contractions):
        self.contractions = contractions
    def set_symbol(self, symbol):
        self.symbol = symbol
    
    
    
    def massDim(self):
        
        counter = 0
        
        for field in self.get_elementary():
            if field.get_symbol() == 'P':
                counter = counter + 1
            elif field.get_symbol() == 'DP':
                counter = counter + 2
        
        return counter                       

    def LORrank(self):
        '''
        To get LORrank Count number of Lorentz contractions (counter) in contractions. Multiply by 2. Subtract this from 
        the number of Lorentz indices appearing in elementary - namely, the number of 'DP' fields. 
        '''
        counter1 = 0
        for pair in self.get_contractions():
            if pair[0] == 1:
                counter1 = counter1 + 1
            else:
                pass
            
        counter2 = 0
        for field in self.get_elementary():
            if field.get_symbol() == 'DP':
                counter2 = counter2 + 1
            else:
                pass     
        #print("counter1: " + str(counter1))
        #print("counter2: " + str(counter2))
        
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def ONrank(self):
        
        counter1 = 0
        
        #Number of contractions is the number of time a '1' appears in the second index of pairs in contractions
        for pair in self.get_contractions():
            if pair[1] == 1:
                counter1 = counter1 + 1
            else:
                pass
        
        #Number of ON indices is just the number of fields in self.elementary
        counter2 = len(self.get_elementary_str())
            
        rank = counter2 - 2*counter1
        
        return rank
    
    
    def contract(self, other, contraction, symbol_prod):
        '''
        other: elementary field, P or DP
        contraction: pair of integers - [1,0] for contraction in LOR, [0,1] for contraction in ON, [1,1] for both
        symbol_prod: string
        '''
        
        #print("contraction: " + str(contraction))
        
        if contraction == [1,0]:
            contractions_prod = self.get_contractions() + [[1,0]]
        elif contraction == [0,1]:
            contractions_prod = self.get_contractions() + [[0,1]]
        elif contraction == [1,1]:
            contractions_prod = self.get_contractions() + [[1,1]]
        else:
            print("Error: contractions must be [1,0], [0,1], or [1,1]")
            
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    def multiply(self, other, symbol_prod=''):
        '''
        Multiplies without contraction
        INPUT:
        self: field object
        other: field object
        symbol_prod: symbol for new field obj output
        
        OUTPUT:
        prod: field object, concatenating self and other, by inserting [0,0] in contractions
        
        '''
        if (not self.get_elementary_str()) or (not other.get_elementary_str()):
            print("one factor is const")
            contractions_prod = self.get_contractions() + other.get_contractions()
        else:
            contractions_prod = self.get_contractions() + [[0,0]] + other.get_contractions()
        
        elementary_str_prod = self.get_elementary_str() + other.get_elementary_str()
        
        prod = field(elementary_str_prod, contractions_prod, symbol_prod)
        
        prod.set_elementary()
        
        return prod
    
    
    
    def generate_connected(self, d):
        '''
        INPUT
        d: non-negative integer, mass dimension up to which to generate field terms
        
        OUTPUT
        terms: list of field terms of mass dimension no larger than d.  
        '''
        
        const = field([],[],'1')
        
        terms = [const]
        
        #print("in generate_connected")
        
        if d >= 2:
            #print("in d>=2")
            field1 = field(['P'],[],'P')
            field2 = field(['P'],[],'P')
            prod = field.contract(field1, field2, [0,1], 'PP')
            terms.append(prod)
        if d >= 4:
            deriv1 = field(['DP'],[],'DP')
            deriv2 = field(['DP'],[],'DP')
            prod = field.contract(deriv1, deriv2, [1,1], 'DPDP')
            terms.append(prod)
        
        field1 = field(['P'],[],'P')
        deriv1 = field(['DP'],[],'DP')
        
        if d < 4:
            return terms
    
        
        root = field.contract(field1, deriv1, [0,1], 'PDP')
        
        while root.massDim() <= d:
            #print("")
            #print("root: ")
            #print(root)
            #print("")
            if root.LORrank() != 0:
                #print("in root.LORrank()!=0")
                deriv1 = field(['DP'],[],'DP')
                root = field.contract(root, deriv1, [1,0], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")
            
            if root.ONrank() != 0:
                #print("in root.ONrank()!=0")
                field1 = field(['P'],[],'P')
                deriv1 = field(['DP'],[],'DP')
                
                new = field.contract(root, field1, [0,1], '')
                if new.massDim() <= d and new.LORrank() == 0 and new.ONrank() == 0:
                    #print("in new term added")
                    terms.append(new)
                
                root = field.contract(root, deriv1, [0,1], '')
                #print("")
                #print("root: ")
                #print(root)
                #print("")    
        
        return terms
    
    
    '''
    def generate(self, D):
        print("In generate, D = " + str(D))
        if D < 2:
            return [field([], [], '')], [field([], [], '')] 
        if D == 2:
            const = field([], [], '')
            PP = field(['P','P'],[[0,1]],'PP')
            return [const, PP], [const, PP] #return generated terms and cache for higher D's
        else:
            connected_terms_D = field.generate_connected(self, D)
            print("")
            print("Check Connected_terms_D: ")
            for term in connected_terms_D:
                print(term)
                print("")
            
            cache = []
            generated = []
            
            for i in range(len(connected_terms_D)):
                for j in range(len(self.generate(D-1)[1])):
                    prod = field.multiply(connected_terms_D[i], self.generate(D-1)[1][j])
                    cache.append(prod)
            
            for term in cache:
                if term.massDim() <= D:
                    generated.append(term)
               
    '''
    
    def generate(self, D):
        if D < 2:
            print("D should be >= 2.")
            return []
        else:
            d = 2
        
            const = field([], [], '')
            PP = field(['P','P'],[[0,1]],'PP')
            cache = [const, PP]
        
            while(d <= D):
                #print(d)
                connected_terms_d = self.generate_connected(d)
                #print("len(connected_terms_d): " + str(len(connected_terms_d)))
                #print("len(cache): " + str(len(cache)))
                
                cache_len = len(cache)
                
                for i in range(len(connected_terms_d)):
                    for j in range(cache_len):
                        print("i,j:" + str(i) + ", " + str(j))
                        prod = field.multiply(connected_terms_d[i], cache[j])
                        cache.append(prod)
            
                d +=1
            
            generated = []
            
            for term in cache:
                if term.massDim() <= D:
                    generated.append(term)
                
            return generated
                    
            
            
            
    
    
            
                    
            
    
                
                
            
        

In [48]:
A = field(['P','DP','DP'],[[0,1],[1,0]],'')

In [49]:
A.generate(6)

2
len(connected_terms_d): 2
len(cache): 2
i,j:0, 0
one factor is const
i,j:0, 1
one factor is const
i,j:1, 0
one factor is const
i,j:1, 1
3
len(connected_terms_d): 2
len(cache): 6
i,j:0, 0
one factor is const
i,j:0, 1
one factor is const
i,j:0, 2
one factor is const
i,j:0, 3
one factor is const
i,j:0, 4
one factor is const
i,j:0, 5
one factor is const
i,j:1, 0
one factor is const
i,j:1, 1
i,j:1, 2
one factor is const
i,j:1, 3
i,j:1, 4
i,j:1, 5
4
len(connected_terms_d): 3
len(cache): 18
i,j:0, 0
one factor is const
i,j:0, 1
one factor is const
i,j:0, 2
one factor is const
i,j:0, 3
one factor is const
i,j:0, 4
one factor is const
i,j:0, 5
one factor is const
i,j:0, 6
one factor is const
i,j:0, 7
one factor is const
i,j:0, 8
one factor is const
i,j:0, 9
one factor is const
i,j:0, 10
one factor is const
i,j:0, 11
one factor is const
i,j:0, 12
one factor is const
i,j:0, 13
one factor is const
i,j:0, 14
one factor is const
i,j:0, 15
one factor is const
i,j:0, 16
one factor is const
i,j:0, 17

i,j:0, 109
one factor is const
i,j:0, 110
one factor is const
i,j:0, 111
one factor is const
i,j:0, 112
one factor is const
i,j:0, 113
one factor is const
i,j:0, 114
one factor is const
i,j:0, 115
one factor is const
i,j:0, 116
one factor is const
i,j:0, 117
one factor is const
i,j:0, 118
one factor is const
i,j:0, 119
one factor is const
i,j:0, 120
one factor is const
i,j:0, 121
one factor is const
i,j:0, 122
one factor is const
i,j:0, 123
one factor is const
i,j:0, 124
one factor is const
i,j:0, 125
one factor is const
i,j:0, 126
one factor is const
i,j:0, 127
one factor is const
i,j:0, 128
one factor is const
i,j:0, 129
one factor is const
i,j:0, 130
one factor is const
i,j:0, 131
one factor is const
i,j:0, 132
one factor is const
i,j:0, 133
one factor is const
i,j:0, 134
one factor is const
i,j:0, 135
one factor is const
i,j:0, 136
one factor is const
i,j:0, 137
one factor is const
i,j:0, 138
one factor is const
i,j:0, 139
one factor is const
i,j:0, 140
one factor is const
i,j:0, 1

In [50]:
B = field(['P','DP','DP'],[[0,1],[1,0]],'PDPDP')

In [51]:
A == B

True

In [59]:
mylist = [A,B]

In [60]:
len(mylist)

2

In [61]:
set(mylist)

TypeError: unhashable type: 'field'